 # How to pull medication data

In [ ]:
# In your DNA Nexus project, use the cohort browser to pull the data for: GP prescription records, Field #: 42039
# This should give you 222,073 individuals 
# These columns: 
#Participant ID
#Data provider
#Date prescription was issued
#Read v2 code
#BNF code
#dm+d code
#Drug name
#Quantity issued
#Export to CSV using table exporter to create file drugs_april_2024_gp_scripts.csv

# Load created csv file created in previous step

In [ ]:
# Imports
import pandas as pd
import numpy as np

In [ ]:
!dx download 'drugs_april_2024_gp_scripts.csv'

In [ ]:
# Load total drug csv table
d = pd.read_csv('drugs_april_2024_gp_scripts.csv')
d

In [ ]:
# Where did data come from? -- be aware of this if comparing with SAIL (data from Wales)
d.data_provider.value_counts()

# Basic cleaning of dataset -- to make sure each entry has a drug_name

In [ ]:
#Drugs that already have a drug_name
d1 = d[~d['drug_name'].isna()]
d1

In [ ]:
# clean up columns
d1 = d1[['eid', 'issue_date', 'drug_name', 'data_provider', 'read_2', 'bnf_code']]
d1 = d1.rename(columns = {'eid':'ID'})
d1

In [ ]:
#drugs without a drug_name
d2 = d[d['drug_name'].isna()]
d2

In [ ]:
#The majority of these appear to be from Wales
d2.data_provider.value_counts()

In [ ]:
#Look at the ones not from Wales
#Omit these people because they have no drug information
test = d2[d2['data_provider'] != 'Wales']
test

In [ ]:
d2 = d2[d2['data_provider'] == 'Wales']
d2

In [ ]:
#How many do not have read_2 codes?
#They all have a read_2 code
d2.read_2.isna().value_counts()

## To find the files referenced below, visit https://biobank.ctsu.ox.ac.uk/crystal/refer.cgi?id=592 and download primarycare_codings.zip

## These files are tabs in the all_lkps_maps_v4.xlsx sheet

In [ ]:
#read_v2_drugs_lkp came from UKB website
!dx download 'read_v2_drugs_lkp.csv'

In [ ]:
#read_v2_drugs_lkp came from UKB website
read2 = pd.read_csv('read_v2_drugs_lkp.csv')
read2

In [ ]:
#Merge code spreadsheet
d2 = d2.merge(read2, left_on = 'read_2', right_on = 'read_code', how = 'left')
d2

In [ ]:
# Rename/reorder columns
d2 = d2[['eid', 'issue_date', 'term_description', 'data_provider', 'read_2', 'bnf_code']]
d2 = d2.rename(columns = {'eid':'ID', 'term_description':'drug_name'})
d2

In [ ]:
# Which samples still have no names?
# We got them all!
test = d2[d2['drug_name'].isna()]
test

In [ ]:
#Create final df
df = pd.concat([d1, d2])
df

In [ ]:
#Final N count: 56,216,345
#N of people with data - 222,073
people = list(set(list(df['ID'])))
print(len(people))

# Remove Scottish non-drug codes

In [ ]:
#Download reference files from UKB
#!dx download 'bnf_lkp.csv'

In [ ]:
bnf = pd.read_csv('bnf_lkp.csv')
bnf = bnf[~bnf['BNF_Presentation_Code'].isna()]
bnf

In [ ]:
#Merge
df = df.merge(bnf, left_on = 'bnf_code', right_on = 'BNF_Presentation_Code', how = 'left')
df

In [ ]:
#Split the 'DUMMY' column
df['dummy'] = df['BNF_Product'].str.split(' ').str[0]
df

In [ ]:
#How many are dummy products? N=8463
test = df[df['dummy'] == 'DUMMY']
test

In [ ]:
#Remove all dummy products from DF -- these are medical prodcuts (like lancets, peak flow meters, etc) not medications
df = df[df['dummy'] != 'DUMMY']
df

In [ ]:
# Select needed columns
df = df[['ID', 'issue_date', 'drug_name', 'data_provider', 'read_2', 'bnf_code']]
df

# Save raw medications dataset -- final_drug_may_2024.csv

In [ ]:
df.to_csv('final_drug_may_2024.csv', header = True, index = False)

In [ ]:
! dx upload 'final_drug_may_2024.csv' --path data/final_drug_may_2024.csv